In [1]:
import openai
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
import requests, uuid, json

In [2]:
openai.api_type = "azure"
openai.api_base = "https://fusion-gpt-training.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "a83037e20d3d4d7982b0c52080e730fb"

In [3]:
host = "pureelawvec.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Puree1234"
sslmode = "require"
# Construct connection string

In [4]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

Connection established


In [19]:
import openai  # Make sure to import the OpenAI library

class Chatbot():
    def __init__(self):
        self.memory = [{"role": "system", "content": ""}, {"role": "user", "content": ""}]
        self.Lawyer_prompt = "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
        Analysis the Question sector and provide a comprehensive and briefly answer in Answer section.The answer must address all aspects\
        of the Question and support your argument with appropriate legal references in Context section. Only answered by thai language only "
    
    def thai2eng(self ,input):
        key = "041e6b79b96146eabafaf524d78911be"
        endpoint = "https://api.cognitive.microsofttranslator.com"
        location = "southeastasia"
        path = '/translate'
        constructed_url = endpoint + path
        params = {
            'api-version': '3.0',
            'from': 'th',
            'to': ['en']
        }
        headers = {
            'Ocp-Apim-Subscription-Key': key,
            'Ocp-Apim-Subscription-Region': location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }
        body = [{
            'text': input
        }]
        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()
        return response[0]["translations"][0]['text']

    def text2vec(self, input):
        embed_list = []
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        embeddings = model.encode(input)
        embed_list.append(embeddings.tolist())
        return embed_list[0]

    def search_vectordb(self, question):
        search_text = []
        try:
            cursor.execute(f"""SELECT text_thai, cosine_distance('{self.text2vec(question)}', embedding) as cosign
            FROM Criminal_Law
            ORDER BY cosine_distance('{self.text2vec(question)}', embedding) ASC LIMIT 7
            """)
            query = cursor.fetchall()
            text_eng_results = [row[0] for row in query] 
            return text_eng_results
        except Exception as e:  
                print(f"Error: {e}")
                conn.rollback()
        return "Not found any related."

    def gpt3_sum(self, user_input, engine='gpt-35-turbo', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
        retry, max_retry = 0, 10
        while retry < max_retry:
            try:
                setting = [{"role": "system", "content": "Summary the user input as a bullet, use only Thai language"}, 
                        {"role": "user", "content": f"{user_input}"}]
                response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
                                                        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen,
                                                        messages=setting)
                return response['choices'][0]['message']['content']
            except openai.error.OpenAIError as e:
                print(f"Error: {e}")
                retry += 1
                if retry < max_retry:
                    print(f"Due to content filtering, retrying. (attempt {retry}/{max_retry})...")
                    time.sleep(3)  # Add a delay before retrying
                else:
                    print("Max retries reached. Exiting. Please change the question")
                    break
        return response['choices'][0]['message']['content']
    def chat(self, user_input, engine='gpt-4-32k', temp=0.5, top_p=0.95, tokens=4000, freq_pen=0.0, pres_pen=0.2):
        retry, max_retry = 0, 10
        while retry < max_retry:
            try:
                # user_input = input("User:")
                # if user_input == "stop":
                #     retry += max_retry
                #     break
                self.clear_memory()
                self.update_memory('user', user_input)
                lawyer_prompt_content = self.Lawyer_prompt + ''.join(self.search_vectordb(self.thai2eng(self.gpt3_sum(user_input))))
                self.update_memory('system', lawyer_prompt_content)
                response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
                                                        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen,
                                                        messages=self.memory)
                # self.update_memory('system', "This is memory section that keep old conversation <memory>{mem}</memory>"
                #     .format(mem = response['choices'][0]['message']['content']))
                # print("LawChatBot:", response['choices'][0]['message']['content'])
                return response['choices'][0]['message']['content']
            except openai.error.OpenAIError as e:
                print(f"Error: {e}")
                retry += 1
                if retry < max_retry:
                    print(f"Due to content filtering, retrying. (attempt {retry}/{max_retry})...")
                    time.sleep(3)  # Add a delay before retrying
                else:
                    print("Max retries reached. Exiting. Please change the question")
                    break
                
    def update_memory(self, role, content):
        self.memory.append({"role": role, "content": content})
        
    def clear_memory(self):
        self.memory = [{"role": "system", "content": ""}, {"role": "user", "content": ""}]


In [20]:
bot = Chatbot() 

In [18]:
#ขโมยของผิดไหม
#ฆ่าคนมีความผิดไหม
bot.chat("ขโมยของมีความผิดไหม")

TypeError: Chatbot.clear_memory() takes 1 positional argument but 2 were given

In [21]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Puree\Desktop\work\Project_law\validation.csv", encoding = "utf-8")
chat_ans = []
for data in range(3):
    x = bot.chat(df.Question.iloc[data])
    chat_ans.append(x)
answer = pd.DataFrame(chat_ans, columns=["Answer"])
answer.to_csv("./answer_test.csv")

In [32]:
conn.commit()
cursor.close()
conn.close()